In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [24]:
CSV_FILE = 'ssh_logs_processed.csv'

print(f"Loading data from {CSV_FILE}...")
try:
    df = pd.read_csv(CSV_FILE)
except FileNotFoundError:
    print("Không tìm thấy file")

df['label'] = df['classification'].map({'bruteforce': 1, 'other': 0})
X = df[['occ_count', 'fail_count']]
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Loading data from ssh_logs_processed.csv...


In [25]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [26]:
model = LogisticRegression(penalty='l2', random_state=42)

print("training")
model.fit(X_train_scaled, y_train)

training


LogisticRegression(random_state=42)

In [27]:
y_pred = model.predict(X_test_scaled)

print("res")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}")
print(classification_report(y_test, y_pred, target_names=['Other', 'Bruteforce']))

res
Accuracy: 1.00
              precision    recall  f1-score   support

       Other       1.00      1.00      1.00        14
  Bruteforce       1.00      1.00      1.00         6

    accuracy                           1.00        20
   macro avg       1.00      1.00      1.00        20
weighted avg       1.00      1.00      1.00        20



In [28]:
def predict_new_ip(occ, fail):
    input_data = pd.DataFrame([[occ, fail]], columns=['occ_count', 'fail_count'])
    input_scaled = scaler.transform(input_data)
#guess
    prob = model.predict_proba(input_scaled)[0][1]
    label = "BRUTEFORCE" if prob > 0.5 else "Normal"

    return prob, label

test_cases = [
    (3000, 2000), # Tấn công rõ ràng
    (5, 0),       # Bình thường
    (100, 51),    # Nghi vấn
]

for occ, fail in test_cases:
    prob, label = predict_new_ip(occ, fail)
    print(f"IP (Occ: {occ}, Fail: {fail}) - Xác suất: {prob*100:.2f}% | {label}")

IP (Occ: 3000, Fail: 2000) - Xác suất: 100.00% | BRUTEFORCE
IP (Occ: 5, Fail: 0) - Xác suất: 24.35% | Normal
IP (Occ: 100, Fail: 51) - Xác suất: 48.76% | Normal
